In [10]:
import os
import pandas as pd
from IPython.display import display, HTML
import re

#path = "c:\\data\\"
path = "/Users/jeff/temp/dissertation"


filename_read = os.path.join(path,"workload.csv")
df = pd.read_csv(filename_read)


# Experiment 1

In [11]:
import re

s = 'neural-r:sqrt-y0|rmse'
result = re.search('\:(.*)\|', s).group(1)
print(result.group(1))

AttributeError: 'str' object has no attribute 'group'

In [12]:
lookup = {'diff-y0' : 'Difference',
'log-y0' : 'Log',
'poly-y0' : 'Polynomial',
'poly2-y0' : 'Polynomial2',
'pow-y0' : 'Power',
'ratio-y0' : 'Ratio',
'ratio_diff-y0' : 'Ratio Difference',
'ratio_poly-y0' : 'Ratio Polynomial',
'ratio_poly2-y0' : 'Ratio Polynomial2',
'sqrt-y0' : 'Square Root'}


ex1 = df[df.name == 'experiment-1'][['dataset','algorithm','predictors','cycle','result','result-raw']]
f = {'result':['min','mean','std']}
ex1_summary = ex1.groupby(['algorithm']).agg(f)

ex1_summary.columns = [x[1] for x in ex1_summary.columns.values]
ex1_summary.insert(0,'algorithm', ex1_summary.index)
ex1_summary.insert(1, 'name' ,[re.search('\:(.*)\|', x).group(1) for x in ex1_summary.algorithm])
ex1_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex1_summary.algorithm])
ex1_summary = ex1_summary[ex1_summary.name.isin(lookup.keys())]
ex1_summary.insert(1, 'expression' ,[lookup[x] for x in ex1_summary.name])
ex1_summary.sort_values(by=['expression'],inplace=True)
ex1_summary.index = range(len(ex1_summary))
ex1_summary.insert(0, 'num', ["{}-{}".format(1,x+1) for x in range(len(ex1_summary))])



In [13]:
# Experiment 1 neural results
ex1_summary_neural = ex1_summary[ex1_summary['set'] == 'neural'][['num','expression','min','mean','std']]
display(ex1_summary_neural)
ex1_summary_neural.to_csv(os.path.join(path,"diss_ex1_neural.csv"),index=False)

,num,expression,min,mean,std
1,1-2,Difference,0.014007,0.381728,0.406555
3,1-4,Log,0.777476,0.777750,0.000212
5,1-6,Polynomial,0.759676,2.952394,1.225767
7,1-8,Polynomial2,0.895603,1.092122,0.109922
9,1-10,Power,0.238305,0.287241,0.027356
11,1-12,Ratio,27.423929,27.455613,0.029165
13,1-14,Ratio Difference,22.976520,22.981816,0.003718
14,1-15,Ratio Polynomial,10.065359,10.194288,0.076443
16,1-17,Ratio Polynomial2,0.066731,0.124259,0.055996
19,1-20,Square Root,0.024773,0.089885,0.086011


In [14]:
# Experiment 1 GP results
ex1_summary_gp = ex1_summary[ex1_summary['set'] == 'gp'][['num','expression','min','mean','std']]
display(ex1_summary_gp)
ex1_summary_gp.to_csv(os.path.join(path,"diss_ex1_gp.csv"),index=False)

,num,expression,min,mean,std
0,1-1,Difference,0.000000e+00,0.114616,0.256290
2,1-3,Log,6.966994e-02,0.494058,0.242687
4,1-5,Polynomial,1.144970e+00,1.508141,0.622293
6,1-7,Polynomial2,1.170128e+00,1.228032,0.051963
8,1-9,Power,0.000000e+00,0.000000,0.000000
10,1-11,Ratio,0.000000e+00,10.996490,15.043914
12,1-13,Ratio Difference,2.298286e+01,22.984583,0.001594
15,1-16,Ratio Polynomial,7.826523e+00,9.770340,1.086627
17,1-18,Ratio Polynomial2,1.293278e-01,0.184874,0.044836
18,1-19,Square Root,6.337065e-07,0.059583,0.033307


# Experiment 2

In [15]:
ex2 = df[df.name == 'experiment-2'][['dataset','algorithm','predictors','cycle','result','result-raw']]
#ex2 = ex2[ex2.dataset != 'feature_eng.csv']

f = {'result-raw':['min','mean','std']}
ex2_summary = ex2.groupby(['algorithm','dataset']).agg(f)

ex2_summary.columns = [x[1] for x in ex2_summary.columns.values]
ex2_summary.insert(0,'algorithm', [x[0] for x in ex2_summary.index])
ex2_summary.insert(0,'dataset', [x[1] for x in ex2_summary.index])
ex2_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex2_summary.dataset])
ex2_summary.insert(0, 'set' , [('gp' if 'gp' in x else 'neural') for x in ex2_summary.algorithm])
ex2_summary.index = range(len(ex2_summary))
ex2_summary.sort_values(by=['set','name'],inplace=True)

ex2_summary_neural = ex2_summary[ex2_summary['set'] == 'neural'][['name','min','mean','std']]
ex2_summary_neural.columns = ['name','neural_min','neural_mean','neural_std']
ex2_summary_gp = ex2_summary[ex2_summary['set'] == 'gp'][['name','min','mean','std']]
ex2_summary_gp.columns = ['name','gp_min','gp_mean','gp_std']

#display(ex2_summary_neural)
#display(ex2_summary_gp)
display(pd.merge(ex2_summary_neural,ex2_summary_gp,how="left"),on=['name'])

,name,neural_min,neural_mean,neural_std,gp_min,gp_mean,gp_std
0,abalone,2.240236,2.890617,0.452490,3.141404,3.225386,0.091805
1,auto-mpg,2.240557,4.419963,2.867133,4.230193,6.335783,1.274852
2,bupa,0.412168,0.458360,0.038773,0.488553,0.537161,0.035475
3,covtype,0.384562,0.399716,0.013042,NaN,NaN,NaN
4,crx,0.345685,0.364244,0.017183,0.391675,0.391675,0.000000
5,forestfires,0.057387,0.060501,0.002381,NaN,NaN,NaN
6,glass,0.318461,0.337616,0.013523,NaN,NaN,NaN
7,heart_cleveland,0.317679,0.331056,0.010689,NaN,NaN,NaN
8,hepatitis,0.445552,0.456697,0.010164,0.447214,0.512942,0.048575
9,horse-colic,0.346672,0.364319,0.017769,NaN,NaN,NaN


# Experiment 3

In [8]:
ex3 = df[df.name == 'experiment-3'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex3_summary = ex3.groupby(['dataset']).agg(f)

ex3_summary.columns = [x[1] for x in ex3_summary.columns.values]
ex3_summary.insert(0,'dataset', ex3_summary.index)
ex3_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex3_summary.dataset])
ex3_summary.index = range(len(ex3_summary))
ex3_summary.drop('dataset', axis=1, inplace=True)
ex3_summary.columns = ['name','ensemble_min','ensemble_mean','ensemble_std']

ex3_summary = pd.merge(ex3_summary,ex2_summary_neural,how="left",on=['name'])
ex3_summary = pd.merge(ex3_summary,ex2_summary_gp,how="left",on=['name'])
ex3_summary.drop(['neural_std','neural_min','gp_std','gp_min'], axis=1, inplace=True)

display(ex3_summary)

,name,ensemble_min,ensemble_mean,ensemble_std,neural_mean,gp_mean
0,abalone,2.415381,2.729983,0.306660,2.572675,1.321847e+00
1,auto-mpg,2.367463,2.520249,0.177967,4.398705,3.790242e+00
2,bupa,0.428034,0.444665,0.015502,0.431751,3.637265e-01
3,crx,0.326418,0.356366,0.023253,0.353303,0.000000e+00
4,hepatitis,0.441576,0.473846,0.027822,0.454582,2.212727e-10
5,pima-indians-diabetes,0.391882,0.401453,0.010405,0.413272,2.864604e-01
6,wcbreast_wdbc,0.080417,0.089454,0.013049,0.095292,2.300154e-01
7,wcbreast_wpbc,0.352044,0.357472,0.006107,0.382045,7.915624e-02


# Experiment 6

In [16]:
ex6 = df[df.name == 'experiment-6'][['dataset','algorithm','predictors','cycle','result','result-raw']]

f = {'result-raw':['min','mean','std']}
ex6_summary = ex6.groupby(['algorithm','dataset']).agg(f)

ex6_summary.columns = [x[1] for x in ex6_summary.columns.values]
ex6_summary.insert(0,'algorithm', [x[0] for x in ex6_summary.index])
ex6_summary.insert(0,'dataset', [x[1] for x in ex6_summary.index])
ex6_summary.insert(1, 'name' ,[re.search('(.*)\.', x).group(1) for x in ex6_summary.dataset])
ex6_summary.index = range(len(ex6_summary))
ex6_summary.drop(['dataset','algorithm'], axis=1, inplace=True)
ex6_summary.columns = ['name','auto_min','auto_mean','auto_std']
ex6_summary = pd.merge(ex6_summary,ex3_summary,how="left",on=['name'])
ex6_summary = pd.merge(ex6_summary,ex2_summary_neural,how="left",on=['name'])
ex6_summary.neural_mean_x.fillna(ex6_summary.neural_mean_y,inplace=True)
ex6_summary.drop(['auto_min','auto_std','ensemble_min','ensemble_std','neural_mean_y','neural_min','neural_std'], 
                 axis=1, inplace=True)
ex6_summary.rename(columns={'neural_mean_x':'neural_mean'},inplace=True)
ex6_summary['best_non_fe'] = ex6_summary[['ensemble_mean','neural_mean','gp_mean']].min(axis=1,skipna=True)
ex6_summary['diff'] = ex6_summary.auto_mean - ex6_summary.best_non_fe
ex6_summary['pct'] = (ex6_summary.auto_mean - ex6_summary.best_non_fe) / ex6_summary.best_non_fe

ex6_summary.sort_values(by=['name'],inplace=True)
ex6_summary.index = range(len(ex6_summary))



display(ex6_summary)

,name,auto_mean,ensemble_mean,neural_mean,gp_mean,best_non_fe,diff,pct
0,abalone,3.199212,2.729983,2.572675,1.321847e+00,1.321847e+00,1.877365,1.420259e+00
1,auto-mpg,2.476317,2.520249,4.398705,3.790242e+00,2.520249e+00,-0.043932,-1.743156e-02
2,bupa,0.479200,0.444665,0.431751,3.637265e-01,3.637265e-01,0.115474,3.174744e-01
3,covtype,0.397740,NaN,0.399716,NaN,3.997163e-01,-0.001976,-4.944735e-03
4,crx,0.342399,0.356366,0.353303,0.000000e+00,0.000000e+00,0.342399,inf
5,forestfires,0.061441,NaN,0.060501,NaN,6.050122e-02,0.000939,1.552551e-02
6,glass,0.364221,NaN,0.337616,NaN,3.376156e-01,0.026605,7.880365e-02
7,heart_cleveland,0.349227,NaN,0.331056,NaN,3.310562e-01,0.018171,5.488828e-02
8,hepatitis,0.447108,0.473846,0.454582,2.212727e-10,2.212727e-10,0.447108,2.020622e+09
9,horse-colic,0.379658,NaN,0.364319,NaN,3.643187e-01,0.015339,4.210286e-02
